In [1]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(user='gpadmin',
                        password='gpadmin',
                        host='srv8-hausma.net.billing.ru',
                        port="5432",
                        database='ndr_spa_db_paas')
df = pd.DataFrame()

with conn.cursor() as cur:

# Просмотрело пользователей
    cur.execute("""
  SELECT custom_parameter.value AS "User",
             t_dashboard_data_spa."timestamp" AS "Date",
             t_dashboard_data_spa.custom_event_id AS "Action",
            t_ua_browser.family as t_ua_browser_family
            FROM metrics.dashboard_data_spa as t_dashboard_data_spa
            INNER JOIN metrics.ua_browser as t_ua_browser ON t_dashboard_data_spa.browser_id = t_ua_browser.id
            JOIN metrics.custom_parameter ON(metrics.custom_parameter.counterid, metrics.custom_parameter.custom_event_parameter_id, metrics.custom_parameter.name) = (t_dashboard_data_spa.counterid,t_dashboard_data_spa.custom_event_parameter_id,'Цвет кнопки')
            WHERE
                 t_dashboard_data_spa.date_timestamp >= 1650326400.0
AND t_dashboard_data_spa.date_timestamp <= 1653004799.999
AND t_dashboard_data_spa.counterid = ANY(ARRAY[28])
            ORDER BY 1, 2, 3
 """) 
    data = cur.fetchall()
    columns = list(cur.description)
    datacolumns=[c[0] for c in columns]
    df = pd.DataFrame(data=data, columns = datacolumns)
    
#df.head(100)  
df

,User,Date,Action
0,43,1643712041,-5874240070922264888
1,43,1643712063,-5874240070922264888
2,43,1643712114,-5874240070922264888
3,43,1643785551,-5874240070922264888
4,43,1644225964,-638393690216945458
...,...,...,...
945,88,1644316164,-1864130230452195184
946,88,1644316180,7365626932152938428
947,88,1644316195,-1864130230452195184
948,88,1644316201,-1864130230452195184


In [2]:
# 1 ШАГ - ВЫГРУЗКА ДАННЫХ

#выгружаем дата сет из csv
# import pandas as pd
# df = pd.read_csv('C:/Users/Maria.Ruchkina/Desktop/Исходные данные2.csv', encoding='windows-1251', sep=';')

# df['Date'] = pd.to_datetime(df['Date'])
# df['Action'] = df['Action'].astype(str)

#сортируем данные сначала по пользователю, зачем, для каждого пользователя - по дате
df=df.sort_values(by=['User', 'Date', 'Action'])

df.reset_index(inplace = True, drop = True)

#добавляем столбцы - Случай
df["Case"] = ''
#добавляем столбцы - Шаг
df["Step"] = ''
#добавляем столбец - Первый раз
df["FirstTime"] = ''

df.to_csv("исходные данные.csv", sep=';', encoding='windows-1251')
df

,User,Date,Action,Case,Step,FirstTime
0,43,1643712041,-5874240070922264888,,,
1,43,1643712063,-5874240070922264888,,,
2,43,1643712114,-5874240070922264888,,,
3,43,1643785551,-5874240070922264888,,,
4,43,1644225964,-638393690216945458,,,
...,...,...,...,...,...,...
945,88,1644316164,-1864130230452195184,,,
946,88,1644316180,7365626932152938428,,,
947,88,1644316195,-1864130230452195184,,,
948,88,1644316201,-1864130230452195184,,,


In [3]:
#создаем список шагов

steps =[-8794557934084286891, -8763321417171493917, 3779695075062561946]


In [4]:
# 2 ШАГ - ДЛЯ СТРОГОГО ПОРЯДКА - РАЗМЕЧАЕМ СОБЫТИЯ В ДАТА СЕТЕ

# caseNumber - счетчик для случаев
caseNumber = 1

# счетчик для шагов
stepNumber = 0

# цикл по шагам воронки
for step in steps:
    print (stepNumber)    
    # цикл по списку событий
    for i, row in df.iterrows():
        
        # работаем с записью только если для нее не определен шаг
        if row['Step'] == '':
            
            # если у записи событие = текущему шагу воронки
            if row['Action'] == step:
                
                # Пропускаем текущее событие, если:
                # Это не первая запись ИЛИ у предыдущей записи тот же юзер, событие и для него определен тот же шаг или шаг не определен
                # Это необходимо, так как если записи о событии от одного и того же пользователя идут подряд, мы считаем это одним и тем же шагом, а также для для того, чтобы не пропустить шаг, если для 2 шагов указано одно и то же событие
                
                #if i==0 or (df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Action'] == step and (df.iloc[i-1]['Step'] == (stepNumber+1) or df.iloc[i-1]['Step'] == '')) == False:
                    
                    # если текущий шаг - первый, фиксируем случай и шаг
                    if stepNumber == 0:
                        # если предыдущий шаг - с таким же Action и User и номером шага и следующий шаг - с другим Action
                        if df.iloc[i-1]['Action']==step and df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Step'] == stepNumber+1 and step != steps[stepNumber+1]:
                                df.at[i, 'Step'] = stepNumber+1
                                df.at[i, 'Case'] = df.iloc[i-1]['Case']
                                df.at[i, 'FirstTime']=False
                        elif i == 0 or df.iloc[i-1]['User'] != row['User'] or df.iloc[i-1]['Action'] != step:
                            df.at[i, 'Step'] = stepNumber+1
                            df.at[i, 'Case'] = caseNumber
                            df.at[i, 'FirstTime']=True
                            caseNumber = caseNumber + 1
                    # если текущий шаг - не первый
                    else:
                        if df.iloc[i-1]['Action']==steps[stepNumber-1] and df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Step']==stepNumber:
                            df.at[i, 'Case'] = df.iloc[i-1]['Case']
                            df.at[i, 'Step'] = stepNumber+1
                            df.at[i, 'FirstTime']=True
                        elif df.iloc[i-1]['Action']==step and df.iloc[i-1]['User'] == row['User'] and df.iloc[i-1]['Step']==stepNumber+1 and ((stepNumber+1) == len(steps) or step != steps[stepNumber+1]):
                            df.at[i, 'Step'] = stepNumber+1
                            df.at[i, 'Case'] = df.iloc[i-1]['Case']
                            df.at[i, 'FirstTime']=False
                       
    stepNumber = stepNumber + 1
                        
df_res=df[df.FirstTime == True]
df.to_csv("размеченные данные.csv", sep=';', encoding='windows-1251')
df

0
1
2


,User,Date,Action,Case,Step,FirstTime
0,43,1643712041,-5874240070922264888,,,
1,43,1643712063,-5874240070922264888,,,
2,43,1643712114,-5874240070922264888,,,
3,43,1643785551,-5874240070922264888,,,
4,43,1644225964,-638393690216945458,,,
...,...,...,...,...,...,...
945,88,1644316164,-1864130230452195184,,,
946,88,1644316180,7365626932152938428,,,
947,88,1644316195,-1864130230452195184,,,
948,88,1644316201,-1864130230452195184,,,


In [5]:
# ФОРМИРУЕМ СЛУЧАИ

cases_list = pd.DataFrame({'Case': df_res.Case.unique()})
cases_list["User"]=''

step_number = 0

for step in steps:
    step_number = step_number + 1
    if step_number != 1:
        cases_list["Diff_Step"+str(step_number)] = ''
    cases_list["Step"+str(step_number)] = ''
    
for i, case in cases_list.iterrows():
    step_number = 0
    for step in steps:
        step_number = step_number + 1
        step_row = df_res[(df_res['Case'] == case['Case']) & (df_res['Step'] == step_number)]
        if step_row.empty == False:
            cases_list.at[i, "Step" + str(step_number)]=step_row.iloc[0].Date
            cases_list.at[i, 'User']=step_row.iloc[0].User
            if step_number != 1:
                cases_list.at[i, "Diff_Step" + str(step_number)] = (cases_list.at[i, "Step" + str(step_number)] - cases_list.at[i, "Step" + str(step_number-1)])
                
cases_list

,Case,User,Step1,Diff_Step2,Step2,Diff_Step3,Step3
0,1,44,1644225976,,,,
1,2,44,1644226020,,,,
2,3,44,1644226122,,,,
3,4,44,1644299498,,,,
4,5,44,1644299576,16139,1644315715,5,1644315720
5,6,44,1644316098,,,,
6,7,44,1644316125,,,,
7,8,44,1644316159,,,,
8,9,44,1644316191,,,,
9,10,44,1644316223,,,,


In [7]:
# ОТОБРАЖАЕМ МЕТРИКИ

step_number = 0

for step in steps:
    step_number = step_number + 1    
    print("Шаг "+ str(step_number)+":")
    step_df=cases_list[cases_list["Step"+str(step_number)] != '']
    print("Количество случаев (по кастомному параметру - CUSTOM_CASES) ="+ str(step_df.shape[0]))
    print("Количество значений кастомных параметров (CUSTOM) = " + str(step_df.User.unique().shape[0]))
    if step_number != 1:
        print("Среднее время после предыдущего шага = " + str(step_df["Diff_Step" + str(step_number)].mean()))
        print("Среднее время после предыдущего шага (медиана) = " + str(step_df["Diff_Step" + str(step_number)].median()))

Шаг 1:
Количество случаев (по кастомному параметру - CUSTOM_CASES) =10
Количество значений кастомных параметров (CUSTOM) = 1
Шаг 2:
Количество случаев (по кастомному параметру - CUSTOM_CASES) =1
Количество значений кастомных параметров (CUSTOM) = 1
Среднее время после предыдущего шага = 16139.0
Среднее время после предыдущего шага (медиана) = 16139.0
Шаг 3:
Количество случаев (по кастомному параметру - CUSTOM_CASES) =1
Количество значений кастомных параметров (CUSTOM) = 1
Среднее время после предыдущего шага = 5.0
Среднее время после предыдущего шага (медиана) = 5.0
